In [ ]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import os

In [ ]:
device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available! Using GPU for training.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU for training.")

In [ ]:
# Load spectrum data from CSV file
ds = pd.read_csv('forgan_real.csv', low_memory = False) 
ds = ds[ds['multiclassification']==3]
labels = ds['multiclassification']
spectrum_data = ds.iloc[:, :-2]

In [ ]:
# Convert data and labels to tensors
train_data_tensor = torch.tensor(spectrum_data.values, dtype=torch.float32)
train_labels_tensor = torch.tensor(labels.values, dtype=torch.float32)

# Create a TensorDataset
dataset = TensorDataset(train_data_tensor, train_labels_tensor)

# Define batch size
batch_size = 6

# Create DataLoader
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Define discriminator class
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(1331, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )
    
    def forward(self, x):
        output = self.model(x)
        return output
    
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 64),  # Нууц давхрагийн хэмжээ нэмэгдүүлж байна
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 512),  # Нэмэлт давхарга
            nn.ReLU(),
            nn.Linear(512, 1331),
        )
        

    def forward(self, x):
        output = self.model(x)
        return output

In [ ]:
# Initialize discriminator and generator
discriminator = Discriminator().to(device)
generator = Generator().to(device)

# Define loss function
loss_function = nn.BCELoss()

lr_discriminator = 0.0002
lr_generator = 0.0001

# Define optimizers for discriminator and generator
optimizer_discriminator = optim.Adam(discriminator.parameters(), lr=lr_discriminator)
optimizer_generator = optim.Adam(generator.parameters(), lr=lr_generator)

In [ ]:
# Training loop
Dis_losses = []
Gen_losses = []

num_epochs = 10000
for epoch in range(num_epochs):
    for n, (real_samples, _) in enumerate(train_loader):
        batch_size = real_samples.size(0)  # Get the actual batch size

        # Move data to GPU
        real_samples = real_samples.to(device)

        # Data for training the discriminator
        real_samples_labels = torch.full((batch_size, 1), 0.9, device=device)  # Label smoothing here
        
        if n % 3 == 0:
            # latent_space_samples = torch.randn((batch_size, 2), device=device)
            latent_space_samples = torch.randn((batch_size, 2), device=device) + 0.05 * torch.randn((batch_size, 2), device=device)  # Дуу чимээг нэмэх

            generated_samples = generator(latent_space_samples)
            # generated_samples_labels = torch.zeros((batch_size, 1), device=device)
            generated_samples_labels = torch.full((batch_size, 1), 0.1, device=device)  # генераторын fake шошгууд
            all_samples = torch.cat((real_samples, generated_samples))
            all_samples_labels = torch.cat((real_samples_labels, generated_samples_labels))

            # Training the discriminator
            discriminator.zero_grad()
            output_discriminator = discriminator(all_samples)
            loss_discriminator = loss_function(output_discriminator, all_samples_labels)
            loss_discriminator.backward()
            optimizer_discriminator.step()
        
        # Data for training the generator
        latent_space_samples = torch.randn((batch_size, 2), device=device)

        # Training the generator
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)  # Same batch size
        output_discriminator_generated = discriminator(generated_samples)
        loss_generator = loss_function(output_discriminator_generated, real_samples_labels)
        loss_generator.backward()
        optimizer_generator.step()
        
        # Show loss
        Dis_losses.append(loss_discriminator.item())
        Gen_losses.append(loss_generator.item())
        if epoch % 100 == 0 and n == len(train_loader) - 1:
            print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
            print(f"Epoch: {epoch} Loss G.: {loss_generator}")


In [ ]:
# Save generated samples to CSV
tensor_data_array = generated_samples.detach().cpu().numpy()
df = pd.DataFrame(tensor_data_array)
df.to_csv('augmented_3.csv')

# Plot and save loss curve
plt.figure(figsize=(6, 5))
plt.title("Loss During Training", fontsize=18)
plt.plot(Dis_losses, label="Discriminator")
plt.plot(Gen_losses, label="Generator")
plt.xlabel("Iterations", fontsize=16)
plt.ylabel("Loss", fontsize=16)
plt.legend()
plt.show()